In [1]:
 #using mystats env
import pandas as pd
import numpy as np
import pingouin as pg
import association_metrics as am
import scipy.stats as stats
from scipy.stats import chisquare
from scipy import stats
import pyreadstat
import statsmodels.api as sm
import warnings
import os
pd.options.display.float_format = '{:.2f}'.format

/Users/stephenwsong/anaconda3/envs/mystats/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#reading data
dfv = pd.read_csv('dfV.csv')
dfc = pd.read_csv('dfC.csv')

In [3]:
#create a new column based on the value of Platform variable
dfv['MediaType'] = dfv['Publisher'].apply(lambda x: 'Hybrid' if x in ['The Washington Post', 'NPR'] else 'Legacy' if x in ['CNN', 'Tampa Bay Times'] else 'SMonly')
dfc['MediaType'] = dfc['Publisher'].apply(lambda x: 'Hybrid' if x in ['The Washington Post', 'NPR'] else 'Legacy' if x in ['CNN', 'Tampa Bay Times'] else 'SMonly')

In [4]:
#create a dataset without duplicates
dfv_Dupeless = dfv.dropna(subset=['Shots'])

In [5]:
#for dataset WITH duplicate, filling na values with the previous value
NAlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Fairness', 'Agenda', 'Media Intergrity', 'Decorum', 'Language', 'Topic', 'Presentation', 'ExternalVidSource']
for i in NAlist:
    dfv[i].fillna(method='ffill', inplace=True)

In [6]:
#reverse coded variables
dfv['Fairness'] = 6-dfv['Fairness']
dfv['Agenda'] = 6-dfv['Agenda']
dfv['Media Intergrity'] = 6-dfv['Media Intergrity']
dfv.rename(columns={'Media Intergrity': 'MediaIntegrity'}, inplace=True)
dfv.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfv['Decorum'] = 6-dfv['Decorum']
dfv['Language'] = 6-dfv['Language']

dfv_Dupeless['Fairness'] = 6 - dfv_Dupeless['Fairness']
dfv_Dupeless['Agenda'] = 6 - dfv_Dupeless['Agenda']
dfv_Dupeless['Media Intergrity'] = 6 - dfv_Dupeless['Media Intergrity']
dfv_Dupeless.rename(columns={'Media Intergrity': 'MediaIntegrity'}, inplace=True)
dfv_Dupeless.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfv_Dupeless['Decorum'] = 6 - dfv_Dupeless['Decorum'] 
dfv_Dupeless['Language'] = 6 - dfv_Dupeless['Language']

dfc.rename(columns={'Likes/Hearts': 'Likes'}, inplace=True)
dfc.rename(columns={'Comment': 'CommentText'}, inplace=True)


/var/folders/p9/s35_kx491l3f1qcxx11y7p280000gn/T/ipykernel_76046/494263232.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfv_Dupeless['Fairness'] = 6 - dfv_Dupeless['Fairness']
/var/folders/p9/s35_kx491l3f1qcxx11y7p280000gn/T/ipykernel_76046/494263232.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfv_Dupeless['Agenda'] = 6 - dfv_Dupeless['Agenda']
/var/folders/p9/s35_kx491l3f1qcxx11y7p280000gn/T/ipykernel_76046/494263232.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [7]:
#recode variables with value labels
dfv_recode = dfv
dfv_recode['Shots'] = dfv['Shots'].replace({1: 'steady', 0: 'not steady', 99: 'NA'})
dfv_recode['Graphics'] = dfv['Graphics'].replace({1: 'animations', 2: 'filters', 3: 'both', 4: 'none', 99: 'NA'})
dfv_recode['EditStyle'] = dfv['EditsStyle'].replace({1: 'clean', 0: 'not clean', 99: 'NA'})
dfv_recode['Broll'] = dfv['Broll'].replace({1: 'yes', 0: 'no', 99: 'NA'})
dfv_recode['NatSound'] = dfv['NatSound'].replace({1: 'yes', 0: 'no', 99: 'NA'})
dfv_recode['Audio'] = dfv['Audio'].replace({1: 'good', 0: 'poor', 99: 'NA'})
dfv_recode['Music'] = dfv['Music'].replace({1: 'includes', 0: 'lacks', 99: 'NA'})
dfv_recode['Narration'] = dfv['Narration'].replace({1: 'includes', 0: 'lacks', 99: 'NA'})
dfv_recode['Humor'] = dfv['Humor'].replace({1: 'yes', 0: 'no', 99: 'NA'})
dfv_recode['Offensive'] = dfv['Offensive'].replace({1: 'not offensive', 2: 'atrocities', 3: 'pornography', 4: 'hateful', 5: 'other', 99: 'NA'})
dfv_recode['Sourcing'] = dfv['Sourcing'].replace({1: 'use', 0: 'not use', 99: 'NA'})
dfv_recode['Topic'] = dfv['Topic'].replace({1: 'West Bank Conflict', 2: 'Political', 3: 'Disaster/Accident', 4: 'Entertainment/Pop Culture', 5: 'Technology/AI', 6: 'Economics', 7: 'Environmental/Health', 8: 'Other'})

dfv_Dupeless_recode = dfv_Dupeless.copy()
dfv_Dupeless_recode['Shots'] = dfv_Dupeless['Shots'].replace({1: 'steady', 0: 'not steady', 99: 'NA'})
dfv_Dupeless_recode['Graphics'] = dfv_Dupeless['Graphics'].replace({1: 'animations', 2: 'filters', 3: 'both', 4: 'none', 99: 'NA'})
dfv_Dupeless_recode['EditStyle'] = dfv_Dupeless['EditsStyle'].replace({1: 'clean', 0: 'not clean', 99: 'NA'})
dfv_Dupeless_recode['Broll'] = dfv_Dupeless['Broll'].replace({1: 'yes', 0: 'no', 99: 'NA'})
dfv_Dupeless_recode['NatSound'] = dfv_Dupeless['NatSound'].replace({1: 'yes', 0: 'no', 99: 'NA'})
dfv_Dupeless_recode['Audio'] = dfv_Dupeless['Audio'].replace({1: 'good', 0: 'poor', 99: 'NA'})
dfv_Dupeless_recode['Music'] = dfv_Dupeless['Music'].replace({1: 'includes', 0: 'lacks', 99: 'NA'})
dfv_Dupeless_recode['Narration'] = dfv_Dupeless['Narration'].replace({1: 'includes', 0: 'lacks', 99: 'NA'})
dfv_Dupeless_recode['Humor'] = dfv_Dupeless['Humor'].replace({1: 'yes', 0: 'no', 99: 'NA'})
dfv_Dupeless_recode['Offensive'] = dfv_Dupeless['Offensive'].replace({1: 'not offensive', 2: 'atrocities', 3: 'pornography', 4: 'hateful', 5: 'other', 99: 'NA'})
dfv_Dupeless_recode['Sourcing'] = dfv_Dupeless['Sourcing'].replace({1: 'use', 0: 'not use', 99: 'NA'})
dfv_Dupeless_recode['Topic'] = dfv_Dupeless['Topic'].replace({1: 'West Bank Conflict', 2: 'Political', 3: 'Disaster/Accident', 4: 'Entertainment/Pop Culture', 5: 'Technology/AI', 6: 'Economics', 7: 'Environmental/Health', 8: 'Other'})


In [8]:
dfv_ColNames = dfv_recode
NAlist2 = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in NAlist2:
    dfv_ColNames.loc[:,i] = dfv_ColNames[i].astype(str) + ' ({})'.format(i)

dfv_Dupeless_ColNames = dfv_Dupeless_recode
NAlist2 = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in NAlist2:
    dfv_Dupeless_ColNames.loc[:,i] = dfv_Dupeless_ColNames[i].astype(str) + ' ({})'.format(i)

In [9]:
#recade variables with value labels
# dfc['Focus'] = dfc['Focus'].replace({1: 'central',  0: 'peripheral', 99: 'NA'})
# change 99 to NA in dfc
dfc = dfc.replace({99: np.nan})

In [10]:

IVlist = ['Platform', 'Publisher', 'MediaType']
DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']

For each DV, define which value is Trendy (1) and which value is Traditional (0).
Add up the variables to create a Composite variable, Trendiness.
You dont have to have all the DVs for the composite variable.
But they have to be 3 or more.

Cronbach's Alpha.
Test for reliability of composite variables
>.80, reliable

In [11]:
# Testing different between Publisher
# Specify the subfolder path
subfolder_path = 'Publisher/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
# Variables to test
DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in DVlist:
    tab1=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'], margins=True)
    tab1.reset_index(inplace=True)
    tab1.columns.name = None
    tab1p=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'], normalize='columns').round(4)*100
    tab1p.reset_index(inplace=True)
    tab1p.columns.name = None
    tab1a=pd.merge(tab1, tab1p, on=i, how='left')
    globals()[f"CrossTab_{i}"] = tab1a
    test=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['Publisher'])# crosstab with raw numbers
    chi2, p, dof, expected = stats.chi2_contingency(test)
    print(i+": " + "chi2 = "+ chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
    # Save the output file in the subfolder
    output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
    tab1a.to_csv(output_file_path, index=False)

Shots: chi2 = 16.554, p = 0.085
Graphics: chi2 = 108.787, p = 0.0
EditsStyle: chi2 = 32.277, p = 0.0
Broll: chi2 = 42.311, p = 0.0
NatSound: chi2 = 38.51, p = 0.0
Audio: chi2 = 10.063, p = 0.073
Music: chi2 = 41.178, p = 0.0
Narration: chi2 = 26.083, p = 0.0
Humor: chi2 = 47.472, p = 0.0
Offensive: chi2 = 14.019, p = 0.172
Sourcing: chi2 = 9.108, p = 0.105
Topic: chi2 = 73.312, p = 0.0


In [12]:
# Testing different between Platform
# Specify the subfolder path
subfolder_path = 'Platform/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
# Variables to test
DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in DVlist:
    tab1=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'], margins=True)
    tab1.reset_index(inplace=True)
    tab1.columns.name = None
    tab1p=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'], normalize='columns').round(4)*100
    tab1p.reset_index(inplace=True)
    tab1p.columns.name = None
    tab1a=pd.merge(tab1, tab1p, on=i, how='left')
    globals()[f"CrossTab_{i}"] = tab1a
    test=pd.crosstab(dfv_ColNames[i], dfv_ColNames['Platform'])# crosstab with raw numbers
    chi2, p, dof, expected = stats.chi2_contingency(test)
    print(i+": " + "chi2 = "+ chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
    # Save the output file in the subfolder
    output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
    tab1a.to_csv(output_file_path, index=False)

Shots: chi2 = 14.209, p = 0.007
Graphics: chi2 = 9.405, p = 0.152
EditsStyle: chi2 = 1.454, p = 0.483
Broll: chi2 = 2.569, p = 0.277
NatSound: chi2 = 2.764, p = 0.251
Audio: chi2 = 0.544, p = 0.762
Music: chi2 = 3.877, p = 0.144
Narration: chi2 = 0.254, p = 0.881
Humor: chi2 = 0.528, p = 0.768
Offensive: chi2 = 12.913, p = 0.012
Sourcing: chi2 = 1.768, p = 0.413
Topic: chi2 = 19.337, p = 0.081


In [13]:
# Testing different between MediaType
# Specify the subfolder path
subfolder_path = 'MediaType/'
# Create the subfolder if it doesn't exist
if not os.path.exists(subfolder_path):
    os.makedirs(subfolder_path)
# Variables to test
DVlist = ['Shots', 'Graphics', 'EditsStyle', 'Broll', 'NatSound', 'Audio', 'Music', 'Narration', 'Humor', 'Offensive', 'Sourcing', 'Topic']
for i in DVlist:
    tab1=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'], margins=True)
    tab1.reset_index(inplace=True)
    tab1.columns.name = None
    tab1p=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'], normalize='columns').round(4)*100
    tab1p.reset_index(inplace=True)
    tab1p.columns.name = None
    tab1a=pd.merge(tab1, tab1p, on=i, how='left')
    globals()[f"CrossTab_{i}"] = tab1a
    test=pd.crosstab(dfv_Dupeless_ColNames[i], dfv_Dupeless_ColNames['MediaType'])# crosstab with raw numbers
    chi2, p, dof, expected = stats.chi2_contingency(test)
    print(i + ": " + "chi2 = " + chi2.round(3).astype(str) + ", p = " + p.round(3).astype(str))
    # Save the output file in the subfolder
    output_file_path = os.path.join(subfolder_path, f"CrossTab_{i}.csv")
    tab1a.to_csv(output_file_path, index=False)

Shots: chi2 = 6.584, p = 0.16
Graphics: chi2 = 67.485, p = 0.0
EditsStyle: chi2 = 3.229, p = 0.199
Broll: chi2 = 4.73, p = 0.094
NatSound: chi2 = 33.459, p = 0.0
Audio: chi2 = 6.205, p = 0.045
Music: chi2 = 0.52, p = 0.771
Narration: chi2 = 2.77, p = 0.25
Humor: chi2 = 35.807, p = 0.0
Offensive: chi2 = 4.068, p = 0.397
Sourcing: chi2 = 2.101, p = 0.35
Topic: chi2 = 36.39, p = 0.0


In [36]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_Dupeless_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,MediaType,2,97,10.93,0.00,0.18
1,Comments,MediaType,2,97,2.69,0.07,0.05
2,MediaIntegrity,MediaType,2,97,12.48,0.00,0.20
3,Presentation,MediaType,2,97,60.28,0.00,0.55


In [15]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,MediaType,Hybrid,Legacy,False,True,0.25,64.33,two-sided,0.80,1.00,bonf,0.257,0.06
1,Likes,MediaType,Hybrid,SMonly,False,True,-3.23,32.51,two-sided,0.00,0.01,bonf,17.872,-0.83
2,Likes,MediaType,Legacy,SMonly,False,True,-3.28,32.62,two-sided,0.00,0.01,bonf,19.693,-0.80
3,Comments,MediaType,Hybrid,Legacy,False,True,-1.28,45.81,two-sided,0.21,0.62,bonf,0.502,-0.33
4,Comments,MediaType,Hybrid,SMonly,False,True,-2.28,45.42,two-sided,0.03,0.08,bonf,2.173,-0.57
5,Comments,MediaType,Legacy,SMonly,False,True,-0.92,59.61,two-sided,0.36,1.00,bonf,0.37,-0.23
6,MediaIntegrity,MediaType,Hybrid,Legacy,False,True,-1.50,63.98,two-sided,0.14,0.42,bonf,0.645,-0.36
7,MediaIntegrity,MediaType,Hybrid,SMonly,False,True,3.50,63.10,two-sided,0.00,0.00,bonf,35.914,0.84
8,MediaIntegrity,MediaType,Legacy,SMonly,False,True,4.77,59.38,two-sided,0.00,0.00,bonf,1428.771,1.19
9,Presentation,MediaType,Hybrid,Legacy,False,True,4.06,64.96,two-sided,0.00,0.00,bonf,173.068,0.97


In [19]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Hybrid,5783.03,8603.68
1,Likes,Legacy,5281.07,7947.59
2,Likes,SMonly,35076.97,50672.91
3,Comments,Hybrid,237.92,528.39
4,Comments,Legacy,466.07,856.32
5,Comments,SMonly,682.12,991.54
6,MediaIntegrity,Hybrid,3.55,0.80
7,MediaIntegrity,Legacy,3.83,0.75
8,MediaIntegrity,SMonly,2.84,0.88
9,Presentation,Hybrid,2.95,0.84


In [20]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_Dupeless_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,Publisher,5,94,4.78,0.00,0.20
1,Comments,Publisher,5,94,3.77,0.00,0.17
2,MediaIntegrity,Publisher,5,94,5.78,0.00,0.24
3,Presentation,Publisher,5,94,27.63,0.00,0.60


In [23]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,Publisher,CNN,GoodMorning BadNews,False,True,-2.10,15.84,two-sided,0.05,0.78,bonf,1.723,-0.70
1,Likes,Publisher,CNN,NPR,False,True,3.46,18.62,two-sided,0.00,0.04,bonf,20.17,1.29
2,Likes,Publisher,CNN,Tampa Bay Times,False,True,4.89,13.01,two-sided,0.00,0.00,bonf,485.818,1.87
3,Likes,Publisher,CNN,The Washington Post,False,True,0.87,31.45,two-sided,0.39,1.00,bonf,0.441,0.28
4,Likes,Publisher,CNN,UnderTheDeskNews,False,True,-1.55,16.03,two-sided,0.14,1.00,bonf,0.843,-0.52
5,Likes,Publisher,GoodMorning BadNews,NPR,False,True,2.76,15.19,two-sided,0.01,0.22,bonf,5.23,0.97
6,Likes,Publisher,GoodMorning BadNews,Tampa Bay Times,False,True,2.95,30.00,two-sided,0.01,0.09,bonf,7.382,1.02
7,Likes,Publisher,GoodMorning BadNews,The Washington Post,False,True,2.30,15.84,two-sided,0.04,0.53,bonf,2.369,0.85
8,Likes,Publisher,GoodMorning BadNews,UnderTheDeskNews,False,True,0.52,30.00,two-sided,0.60,1.00,bonf,0.374,0.18
9,Likes,Publisher,NPR,Tampa Bay Times,False,True,2.28,16.04,two-sided,0.04,0.55,bonf,2.26,0.75


In [31]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['Likes', 'Comments', 'MediaIntegrity', 'Presentation']
DF = dfv_ColNames
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)

In [33]:
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,Platform,2,123,3.50,0.03,0.05
1,Comments,Platform,2,123,4.06,0.02,0.06
2,MediaIntegrity,Platform,2,123,0.24,0.78,0.00
3,Presentation,Platform,2,123,0.07,0.94,0.00


In [34]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,Platform,Instagram Reels,TikTok,False,True,-0.27,82.00,two-sided,0.79,1.00,bonf,0.235,-0.06
1,Likes,Platform,Instagram Reels,YouTube Shorts,False,True,1.96,82.00,two-sided,0.05,0.16,bonf,1.199,0.42
2,Likes,Platform,TikTok,YouTube Shorts,False,True,4.14,82.00,two-sided,0.00,0.00,bonf,263.434,0.90
3,Comments,Platform,Instagram Reels,TikTok,False,True,0.69,82.00,two-sided,0.49,1.00,bonf,0.28,0.15
4,Comments,Platform,Instagram Reels,YouTube Shorts,False,True,2.55,82.00,two-sided,0.01,0.04,bonf,3.649,0.55
5,Comments,Platform,TikTok,YouTube Shorts,False,True,3.06,82.00,two-sided,0.00,0.01,bonf,12.09,0.66
6,MediaIntegrity,Platform,Instagram Reels,TikTok,False,True,0.68,82.00,two-sided,0.50,1.00,bonf,0.278,0.15
7,MediaIntegrity,Platform,Instagram Reels,YouTube Shorts,False,True,0.43,82.00,two-sided,0.66,1.00,bonf,0.247,0.09
8,MediaIntegrity,Platform,TikTok,YouTube Shorts,False,True,-0.24,82.00,two-sided,0.81,1.00,bonf,0.234,-0.05
9,Presentation,Platform,Instagram Reels,TikTok,False,True,-0.09,82.00,two-sided,0.93,1.00,bonf,0.228,-0.02


In [35]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Instagram Reels,26196.71,82502.02
1,Likes,TikTok,30128.95,45105.94
2,Likes,YouTube Shorts,1201.67,3358.39
3,Comments,Instagram Reels,884.40,1700.43
4,Comments,TikTok,679.00,911.47
5,Comments,YouTube Shorts,188.24,496.62
6,MediaIntegrity,Instagram Reels,3.50,1.06
7,MediaIntegrity,TikTok,3.36,0.85
8,MediaIntegrity,YouTube Shorts,3.40,0.94
9,Presentation,Instagram Reels,3.14,1.39


In [40]:
# Define the IV and DV variables
IV = 'Platform'
DV = ['Likes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
merged_anova = pd.DataFrame()
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova


,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,Platform,2,992,10.65,0.00,0.02
1,CharacterNum,Platform,2,988,45.61,0.00,0.08
2,CommLength,Platform,2,993,11.85,0.00,0.02
3,Sentiment,Platform,2,976,5.36,0.00,0.01
4,Focus,Platform,2,991,7.60,0.00,0.02
5,Alignment,Platform,2,537,2.88,0.06,0.01


In [28]:
# Define the IV and DV variables
IV = 'MediaType'
DV = ['Likes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,MediaType,2,992,3.30,0.04,0.01
1,CharacterNum,MediaType,2,988,1.38,0.25,0.00
2,CommLength,MediaType,2,993,1.33,0.26,0.00
3,Sentiment,MediaType,2,976,16.38,0.00,0.03
4,Focus,MediaType,2,991,1.02,0.36,0.00
5,Alignment,MediaType,2,537,16.72,0.00,0.06


In [38]:
merged_pairwise

,Variable,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,p-corr,p-adjust,BF10,hedges
0,Likes,Platform,Instagram Reels,TikTok,False,True,-2.62,500.04,two-sided,0.01,0.03,bonf,2.274,-0.19
1,Likes,Platform,Instagram Reels,YouTube Shorts,False,True,5.02,462.62,two-sided,0.00,0.00,bonf,1.683e+04,0.32
2,Likes,Platform,TikTok,YouTube Shorts,False,True,4.94,386.15,two-sided,0.00,0.00,bonf,1.119e+04,0.32
3,CharacterNum,Platform,Instagram Reels,TikTok,False,True,10.64,491.08,two-sided,0.00,0.00,bonf,3.592e+21,0.74
4,CharacterNum,Platform,Instagram Reels,YouTube Shorts,False,True,3.64,431.34,two-sided,0.00,0.00,bonf,55.852,0.31
5,CharacterNum,Platform,TikTok,YouTube Shorts,False,True,-3.49,241.25,two-sided,0.00,0.00,bonf,33.769,-0.37
6,CommLength,Platform,Instagram Reels,TikTok,False,True,-4.83,739.36,two-sided,0.00,0.00,bonf,6401.314,-0.34
7,CommLength,Platform,Instagram Reels,YouTube Shorts,False,True,-1.32,504.73,two-sided,0.19,0.56,bonf,0.219,-0.11
8,CommLength,Platform,TikTok,YouTube Shorts,False,True,2.92,399.71,two-sided,0.00,0.01,bonf,5.919,0.26
9,Sentiment,Platform,Instagram Reels,TikTok,False,True,-3.07,761.73,two-sided,0.00,0.01,bonf,8.014,-0.22


In [39]:
merged_mean_std_table

,Variable,Value,Mean,Standard Deviation
0,Likes,Instagram Reels,163.13,489.28
1,Likes,TikTok,331.26,1154.47
2,Likes,YouTube Shorts,35.68,101.81
3,CharacterNum,Instagram Reels,123.51,119.41
4,CharacterNum,TikTok,56.34,39.64
5,CharacterNum,YouTube Shorts,86.25,122.41
6,CommLength,Instagram Reels,2.14,0.87
7,CommLength,TikTok,2.40,0.65
8,CommLength,YouTube Shorts,2.23,0.74
9,Sentiment,Instagram Reels,2.61,1.11


In [29]:
# Define the IV and DV variables
IV = 'Publisher'
DV = ['Likes', 'CharacterNum', 'CommLength', 'Sentiment', 'Focus', 'Alignment']
DF = dfc
# Run the ANOVA
for i in DV:
    anova_result = pg.anova(data=DF, dv=i, between=IV)
    globals()[f"ANOVA_{i}"] = anova_result
    globals()[f"ANOVA_{i}"].insert(0, 'Variable', i)
    pairwise_comparisons = pg.pairwise_tests(data=DF, dv=i, between=IV, padjust='bonf')
    globals()[f"Pairwise_{i}"] = pairwise_comparisons
    globals()[f"Pairwise_{i}"].insert(0, 'Variable', i)
    mean_table = DF.groupby(IV)[i].mean().reset_index()
    std_table = DF.groupby(IV)[i].std().reset_index()
    mean_std_table = pd.merge(mean_table, std_table, on=IV)
    mean_std_table.columns = [i, 'Mean', 'Standard Deviation']
    mean_std_table = mean_std_table.rename(columns={i: 'Value'})
    globals()[f"mean_std_table_{i}"] = mean_std_table
    globals()[f"mean_std_table_{i}"].insert(0, 'Variable', i)
merged_anova = pd.concat([globals()[f"ANOVA_{i}"] for i in DV], axis=0)
merged_anova.reset_index(drop=True, inplace=True)
merged_pairwise = pd.concat([globals()[f"Pairwise_{i}"] for i in DV], axis=0)
merged_pairwise.reset_index(drop=True, inplace=True)
merged_mean_std_table = pd.concat([globals()[f"mean_std_table_{i}"] for i in DV], axis=0)
merged_mean_std_table.reset_index(drop=True, inplace=True)
merged_anova.to_csv(IV+'/merged_anova.csv', index=False)
merged_pairwise.to_csv(IV+'/merged_pairwise.csv', index=False)
merged_mean_std_table.to_csv(IV+'/merged_mean_std.csv', index=False)
merged_anova

,Variable,Source,ddof1,ddof2,F,p-unc,np2
0,Likes,Publisher,5,989,11.49,0.00,0.05
1,CharacterNum,Publisher,5,985,1.12,0.35,0.01
2,CommLength,Publisher,5,990,0.70,0.63,0.00
3,Sentiment,Publisher,5,973,13.37,0.00,0.06
4,Focus,Publisher,5,988,3.91,0.00,0.02
5,Alignment,Publisher,5,534,8.56,0.00,0.07
